In [53]:
#Librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import contextlib

import sys
sys.path.append('/home/onyxia/work/Macroeconometrics')
from src.preprocessing import apply_transformation

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

In [54]:
#Main data
fred_md = pd.read_csv("/home/onyxia/work/Macroeconometrics/data/fred_md_2024_12.csv")
#Metadata
fred_info = pd.read_csv("/home/onyxia/work/Macroeconometrics/data/FRED_MD_updated_appendix.csv", encoding="latin1")
#Recession variable
us_rec = pd.read_csv("/home/onyxia/work/Macroeconometrics/data/USREC.csv")

In [55]:
#Indexing the dataset with dates
fred_md_short = (
    fred_md.iloc[1:]
    .assign(sasdate=pd.to_datetime(fred_md.iloc[1:].sasdate, format="%m/%d/%Y"))
    .set_index("sasdate")
)

#Transformation of series based on metadata
for _, row in fred_info.iterrows():
    series_name = row['fred']
    transformation_code = row['tcode']

    with contextlib.suppress(Exception):
        fred_md_short[series_name] = apply_transformation(fred_md_short[series_name], transformation_code)

#Filtering data by date
start_date = "1960"
end_date = "2024"
fred_md_short = fred_md_short[
    (fred_md_short.index >= start_date) & (fred_md_short.index <= end_date)
].dropna(axis=1)

#Addition of the variable of interest (American recession)
us_rec = us_rec.assign(
    observation_date=pd.to_datetime(us_rec.observation_date)
).set_index("observation_date")
us_rec = us_rec.loc[fred_md_short.index,:]

In [56]:
#Function Random Forest
def run_random_forest(X, y, param_grid=None, test_size=0.2, random_state=667, cv_folds=5):
    """
    Perform Random Forest classification with hyperparameter tuning using GridSearchCV.

    Parameters:
    X : Covariates (features) for training the model.
    y : Target variable for classification.
    param_grid : Dictionary of hyperparameters to tune.
    test_size : Fraction of data to use for testing (default is 0.2).
    random_state : Random seed for reproducibility (default is 42).
    cv_folds : Number of folds for cross-validation (default is 5).
    
    Returns:
    dict: Contains 'best_params', 'classification_report', 'accuracy', and 'feature_importance'.
    """
    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split the data into train and test sets (time-series split without shuffling)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, shuffle=False, random_state=random_state)
    
    # Instantiate Random Forest model
    rf = RandomForestClassifier(random_state=random_state)
    
    # Hyperparameter tuning using GridSearchCV
    if param_grid is None:
        param_grid = {
        'n_estimators': [1000],  # Nombre d'arbres dans la forêt
        'max_depth': [None, 10, 30],   # Profondeur maximale des arbres
        'min_samples_split': [2, 5, 10],   # Nombre minimal d'échantillons pour diviser un noeud
        'min_samples_leaf': [1, 4],     # Nombre minimal d'échantillons par feuille
        }
    
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv_folds, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    
    #Best model
    best_rf = grid_search.best_estimator_

    #Predictions on the test set
    y_pred = best_rf.predict(X_test)

    #Evaluation
    classification_rep = classification_report(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    #Feature importance
    importances = best_rf.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': X.columns if isinstance(X, pd.DataFrame) else [f'PC{i+1}' for i in range(X.shape[1])],
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)
    
    # Return results
    return {
        'best_params': grid_search.best_params_,
        'classification_report': classification_rep,
        'accuracy': accuracy,
        'feature_importance': feature_importance_df
    }

# RF naive

In [57]:
# Nous allons créer des variables à t-1 pour chaque série temporelle
fred_md_lagged = fred_md_short.shift(1)

# Fusionner les données laggées avec l'indicateur de récession (us_rec) à l'index
data = pd.concat([fred_md_lagged, us_rec], axis=1)

# Définir les variables X (features) et y (target)
X = data.drop(columns=['USREC'])  # Tout sauf 'us_rec' sera utilisé comme caractéristiques
y = data['USREC']  # La variable cible est 'us_rec'

# Run Random Forest with the custom function
results = run_random_forest(X, y)

# Display the results for 60% explained variance
print("Best Hyperparameters :", results['best_params'])
print("\nClassification Report:")
print(results['classification_report'])
print("\nAccuracy:", results['accuracy'])
print("\nFeature Importance:")
print(results['feature_importance'])

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Hyperparameters : {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1000}

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       152
           1       0.50      0.50      0.50         2

    accuracy                           0.99       154
   macro avg       0.75      0.75      0.75       154
weighted avg       0.99      0.99      0.99       154


Accuracy: 0.987012987012987

Feature Importance:
       Feature  Importance
36     DMANEMP    0.058128
31      PAYEMS    0.055808
32      USGOOD    0.055329
35      MANEMP    0.049528
15   IPMANSICS    0.034101
..         ...         ...
21     CLF16OV    0.000994
106   CPIULFSL    0.000981
24    UEMPMEAN    0.000878
68      REALLN    0.000846
97   OILPRICEx    0.000520

[119 rows x 2 columns]


Problèmes :
- Récessions rares dans le dataset, donc le modèle a tout intérêt à prédire 0 s'il n'a pas assez d'informations sur l'état de l'économie
- Trop de variables (besoin de PCA)
- Besoin d'informations sur les variables en t-2...

# RF with principal components

In [68]:
#data
# Make sure the file paths are correct where you saved the PCA datasets
pca_60_df = pd.read_csv('/home/onyxia/work/Macroeconometrics/data/PCA/pca_60.csv', index_col='sasdate')  # For 60% variance explained
pca_80_df = pd.read_csv('/home/onyxia/work/Macroeconometrics/data/PCA/pca_80.csv', index_col='sasdate')  # For 80% variance explained
pca_90_df = pd.read_csv('/home/onyxia/work/Macroeconometrics/data/PCA/pca_90.csv', index_col='sasdate')  # For 90% variance explained
pca_60_df.index = pd.to_datetime(pca_60_df.index)
pca_80_df.index = pd.to_datetime(pca_80_df.index)
pca_90_df.index = pd.to_datetime(pca_90_df.index)

pca_60_df = pca_60_df.shift(1)
pca_80_df = pca_80_df.shift(1)
pca_90_df = pca_90_df.shift(1)

pca_60_df = pd.concat([pca_60_df, us_rec], axis=1, join='inner').dropna()
pca_80_df = pd.concat([pca_80_df, us_rec], axis=1, join='inner').dropna()
pca_90_df = pd.concat([pca_90_df, us_rec], axis=1, join='inner').dropna()

# Target variable is 'USREC', the recession indicator
X_60 = pca_60_df.drop(columns=['USREC'])
y_60 = pca_60_df['USREC']

X_80 = pca_80_df.drop(columns=['USREC'])
y_80 = pca_80_df['USREC']

X_90 = pca_90_df.drop(columns=['USREC'])
y_90 = pca_90_df['USREC']

In [69]:
# Run Random Forest with the custom function
results_60 = run_random_forest(X_60, y_60)

# Display the results for 60% explained variance
print("Best Hyperparameters for 60% Variance Explained:", results_60['best_params'])
print("\nClassification Report for 60% Variance Explained:")
print(results_60['classification_report'])
print("\nAccuracy for 60% Variance Explained:", results_60['accuracy'])
print("\nFeature Importance for 60% Variance Explained:")
print(results_60['feature_importance'])

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Hyperparameters for 60% Variance Explained: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1000}

Classification Report for 60% Variance Explained:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       152
           1       0.33      0.50      0.40         2

    accuracy                           0.98       154
   macro avg       0.66      0.74      0.70       154
weighted avg       0.98      0.98      0.98       154


Accuracy for 60% Variance Explained: 0.9805194805194806

Feature Importance for 60% Variance Explained:
   Feature  Importance
0      PC1    0.369943
4      PC5    0.132299
3      PC4    0.114240
6      PC7    0.064705
5      PC6    0.063077
7      PC8    0.049956
10    PC11    0.049557
1      PC2    0.038172
2      PC3    0.035285
8      PC9    0.031370
11    PC12    0.028793
9     PC10    0.022604


In [70]:
# Run Random Forest with the custom function
results_80 = run_random_forest(X_80, y_80)

# Display the results for 80% explained variance
print("Best Hyperparameters for 80% Variance Explained:", results_80['best_params'])
print("\nClassification Report for 80% Variance Explained:")
print(results_80['classification_report'])
print("\nAccuracy for 80% Variance Explained:", results_80['accuracy'])
print("\nFeature Importance for 80% Variance Explained:")
print(results_80['feature_importance'])

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Hyperparameters for 80% Variance Explained: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1000}

Classification Report for 80% Variance Explained:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       152
           1       0.33      0.50      0.40         2

    accuracy                           0.98       154
   macro avg       0.66      0.74      0.70       154
weighted avg       0.98      0.98      0.98       154


Accuracy for 80% Variance Explained: 0.9805194805194806

Feature Importance for 80% Variance Explained:
   Feature  Importance
0      PC1    0.302469
4      PC5    0.100981
3      PC4    0.093508
6      PC7    0.045948
5      PC6    0.045179
7      PC8    0.036975
10    PC11    0.036197
1      PC2    0.029107
2      PC3    0.023551
18    PC19    0.020423
16    PC17    0.018347
8      PC9    0.018067
11    PC12    0.017

In [71]:
# Run Random Forest with the custom function
results_90 = run_random_forest(X_90, y_90)

# Display the results for 90% explained variance
print("Best Hyperparameters for 90% Variance Explained:", results_90['best_params'])
print("\nClassification Report for 90% Variance Explained:")
print(results_90['classification_report'])
print("\nAccuracy for 90% Variance Explained:", results_90['accuracy'])
print("\nFeature Importance for 90% Variance Explained:")
print(results_90['feature_importance'])

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Hyperparameters for 90% Variance Explained: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1000}

Classification Report for 90% Variance Explained:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       152
           1       0.25      0.50      0.33         2

    accuracy                           0.97       154
   macro avg       0.62      0.74      0.66       154
weighted avg       0.98      0.97      0.98       154


Accuracy for 90% Variance Explained: 0.974025974025974

Feature Importance for 90% Variance Explained:
   Feature  Importance
0      PC1    0.261004
3      PC4    0.090884
4      PC5    0.090346
5      PC6    0.039208
6      PC7    0.038643
7      PC8    0.031387
10    PC11    0.030516
1      PC2    0.024664
2      PC3    0.019175
18    PC19    0.017811
11    PC12    0.015678
37    PC38    0.015400
43    PC44    0.0152

# RF avec PCA pour 60% en dupliquant les périodes en récession 

In [72]:
# Fonction pour dupliquer les lignes avec une condition
def duplicate_recession(data, factor=2):
    # Sélectionner les lignes où 'USREC' == 1 (récession)
    recession_data = data[data['USREC'] == 1]
    # Sélectionner les lignes où 'USREC' == 0 (non-récession)
    non_recession_data = data[data['USREC'] == 0]
    
    # Dupliquer les lignes de récession par un facteur donné
    duplicated_recession = pd.concat([recession_data] * factor, axis=0, ignore_index=True)
    
    # Concatenner les récessions dupliquées avec les périodes non-récession
    new_data = pd.concat([non_recession_data, duplicated_recession], axis=0, ignore_index=True)
    
    return new_data

# Appliquer la duplication des récessions par un facteur de 2, 5 et 10
data_60_dup_2x = duplicate_recession(pca_60_df, factor=2)
data_60_dup_5x = duplicate_recession(pca_60_df, factor=5)
data_60_dup_10x = duplicate_recession(pca_60_df, factor=10)

# Afficher la taille des nouveaux DataFrames pour vérifier
print("Shape of data_60_dup_2x:", data_60_dup_2x.shape)
print("Shape of data_60_dup_5x:", data_60_dup_5x.shape)
print("Shape of data_60_dup_10x:", data_60_dup_10x.shape)

Shape of data_60_dup_2x: (863, 13)
Shape of data_60_dup_5x: (1148, 13)
Shape of data_60_dup_10x: (1623, 13)


In [73]:
# Now we will use the `run_random_forest` function on the modified datasets
# Prepare data for 60% explained variance (after duplication)
X_60_dup_2x = data_60_dup_2x.drop(columns=['USREC'])
y_60_dup_2x = data_60_dup_2x['USREC']
results_60_dup_2x = run_random_forest(X_60_dup_2x, y_60_dup_2x)

X_60_dup_5x = data_60_dup_5x.drop(columns=['USREC'])
y_60_dup_5x = data_60_dup_5x['USREC']
results_60_dup_5x = run_random_forest(X_60_dup_5x, y_60_dup_5x)

X_60_dup_10x = data_60_dup_10x.drop(columns=['USREC'])
y_60_dup_10x = data_60_dup_10x['USREC']
results_60_dup_10x = run_random_forest(X_60_dup_10x, y_60_dup_10x)

# Displaying the results
print("\nResults for 2x duplicated recession periods:")
print("Best Hyperparameters:", results_60_dup_2x['best_params'])
print("Classification Report:\n", results_60_dup_2x['classification_report'])
print("Accuracy:", results_60_dup_2x['accuracy'])

print("\nResults for 5x duplicated recession periods:")
print("Best Hyperparameters:", results_60_dup_5x['best_params'])
print("Classification Report:\n", results_60_dup_5x['classification_report'])
print("Accuracy:", results_60_dup_5x['accuracy'])

print("\nResults for 10x duplicated recession periods:")
print("Best Hyperparameters:", results_60_dup_10x['best_params'])
print("Classification Report:\n", results_60_dup_10x['classification_report'])
print("Accuracy:", results_60_dup_10x['accuracy'])


Fitting 5 folds for each of 18 candidates, totalling 90 fits


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits

Results for 2x duplicated recession periods:
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1000}
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.03      0.06       173

    accuracy                           0.03       173
   macro avg       0.50      0.01      0.03       173
weighted avg       1.00      0.03      0.06       173

Accuracy: 0.028901734104046242

Results for 5x duplicated recession periods:
Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00       230

    accuracy                           1.00      